In [1]:
# importing packages
from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# loforest and locart functions
from CP2LFI.loforest import ConformalLoforest
from CP2LFI.scores import LambdaScore

from clover import Scores
from clover import LocartSplit

from copy import deepcopy
from tqdm import tqdm

from scipy import stats
from scipy.optimize import minimize_scalar

import time

\section{Comparing our methods with quantile regression in a GMM setting}
Taking $\theta \in \Theta = [0,5]$ we consider a one-dimensional Normal mixture with unknown mean but known unit variance:
\begin{align*}
X \sim 0.5 N(\theta, 1) + 0.5 N(-\theta, 1)
\end{align*}
We consider three different sample sizes:
$$ N = 10, 100, 1000 $$


In [2]:
# simulator
def sim_gmm(n, theta):
    group = np.random.binomial(n=1, p=0.5, size=n)
    X = (((group == 0)*(np.random.normal(theta, 1, size = n))) + 
    ((group == 1)*(np.random.normal(-theta, 1, size = n))))
    return X

# randomly sampling from gmm
def sample_gmm(n, N, seed = 450):
    np.random.seed(seed)
    thetas = np.random.uniform(0, 5, size = n)
    lambdas = np.zeros(n)
    i = 0 
    for theta in tqdm(thetas, desc = "Simulating lambdas and thetas"):
        X = sim_gmm(N, theta)
        lambdas[i] = compute_lrt_statistic(theta, X)
        i += 1
    return thetas, lambdas


# likelihood function
def l_func(theta, x):
    # prob from X_i
    p_xi = np.zeros(x.shape[0])
    i = 0
    for x_i in x:
        p_xi = np.log((0.5*stats.norm.pdf(x_i, loc = theta, scale = 1)) + (0.5*stats.norm.pdf(x_i, loc = -theta, scale = 1)))
        i += 1
    return -(np.sum(p_xi))

# likelihood ratio statistic
def compute_lrt_statistic(theta_0, X):
    # computing MLE
    res = minimize_scalar(l_func, args = (X), bounds = (0, 5)) 
    mle_theta = res.x
    lrt_stat = -2*((-l_func(theta_0, X)) - (-l_func(mle_theta, X)))
    return lrt_stat

# naive method
def naive(alpha, n_grid = 250, B = 1000, N = 100, lower = 0, upper = 5, seed = 250):
    np.random.seed(seed)
    thetas = np.linspace(lower, upper, n_grid)
    quantiles = {}
    for theta in tqdm(thetas, desc = "Obtaining naive quantiles"):
        LRT_stat = np.zeros(B)
        for i in range(0, B):
            X = sim_gmm(N, theta)
            LRT_stat[i] = compute_lrt_statistic(theta, X)
        quantiles[theta] = np.quantile(LRT_stat, q = 1 - alpha)
    return quantiles

# naive predict function
def predict_naive_quantile(theta_grid, quantiles_dict):
    thetas_values = np.array(list(quantiles_dict.keys()))
    quantiles_list = []
    for theta in theta_grid:
        idx = thetas_values[int(np.argmin(np.abs(theta - thetas_values)))]
        quantiles_list.append(quantiles_dict[idx])
    return quantiles_list

Function to evaluate coverage for a given sample size $N$:


In [3]:
def evaluate_coverage(quantiles_dict, thetas, alpha=0.05, n=500, N=100):
    err_data = np.zeros((thetas.shape[0], 4))
    coverage_data = np.zeros((thetas.shape[0], 4))
    j = 0
    for theta in tqdm(thetas, desc="Computing coverage for each method"):
        # generating several lambdas
        LRT_stat = np.zeros(n)
        for i in range(0, n):
            samples = sim_gmm(N, theta)
            LRT_stat[i] = compute_lrt_statistic(theta, samples)

        # comparing coverage of methods
        locart_cover = np.mean(LRT_stat <= quantiles_dict["locart"][j])
        loforest_cover = np.mean(LRT_stat <= quantiles_dict["loforest"][j])
        boosting_cover = np.mean(LRT_stat <= quantiles_dict["boosting"][j])
        naive_cover = np.mean(LRT_stat <= quantiles_dict["naive"][j])

        # appending the errors
        err_locart = np.abs(locart_cover - (1 - alpha))
        err_loforest = np.abs(loforest_cover - (1 - alpha))
        err_boosting = np.abs(boosting_cover - (1 - alpha))
        err_naive = np.abs(naive_cover - (1 - alpha))

        # saving in numpy array
        err_data[j, :] = np.array([err_locart, err_loforest, err_boosting, err_naive])
        coverage_data[j, :] = np.array(
            [locart_cover, loforest_cover, boosting_cover, naive_cover]
        )
        j += 1

    # obtaining MAE and standard error for each method
    mae_vector, std_vector = np.mean(err_data, axis=0), np.std(err_data, axis=0)
    stats_data = pd.DataFrame(
        {
            "methods": ["LOCART", "LOFOREST", "boosting", "naive"],
            "MAE": mae_vector,
            "str": std_vector / (np.sqrt(thetas.shape[0])),
        }
    )

    coverage_data = pd.DataFrame(
        {
            "thetas": thetas,
            "LOCART": coverage_data[:, 0],
            "LOFOREST": coverage_data[:, 1],
            "boosting": coverage_data[:, 2],
            "naive": coverage_data[:, 3],
        }
    )
    return [stats_data, coverage_data]

Now, we compare all methods fixing $\alpha = 0.05$, $N = 100$ and $B = 1000$ for locart, loforest and boosting and n_grid = 30 for naive. Starting by fitting naive:

In [4]:
# running naive
N, B, alpha = 100, 1000, 0.05

start_time = time.time()
naive_quantiles = naive(alpha, n_grid = 30)
end_time = time.time()

running_time = end_time - start_time
print(f"Naive running time: {running_time} seconds.")

Obtaining naive quantiles:   0%|          | 0/30 [00:00<?, ?it/s]

Obtaining naive quantiles:   3%|▎         | 1/30 [03:41<1:47:02, 221.48s/it]

Obtaining naive quantiles:   7%|▋         | 2/30 [07:33<1:46:22, 227.96s/it]

Obtaining naive quantiles:  10%|█         | 3/30 [11:20<1:42:22, 227.51s/it]

Obtaining naive quantiles:  13%|█▎        | 4/30 [14:53<1:36:05, 221.76s/it]

...

Obtaining naive quantiles:  93%|█████████▎| 28/30 [1:06:50<04:28, 134.15s/it]

Obtaining naive quantiles:  97%|█████████▋| 29/30 [1:09:56<02:29, 149.83s/it]

Obtaining naive quantiles: 100%|██████████| 30/30 [1:13:20<00:00, 165.99s/it]

Obtaining naive quantiles: 100%|██████████| 30/30 [1:13:20<00:00, 146.69s/it]

Naive running time: 4400.668137788773 seconds.


Simulating $\theta$ and $\lambda$ to fit the models:


In [5]:
thetas, model_lambdas = sample_gmm(n = B, N = N)
model_thetas = thetas.reshape(-1, 1)

Simulating lambdas and thetas:   0%|          | 0/1000 [00:00<?, ?it/s]

Simulating lambdas and thetas:   0%|          | 2/1000 [00:00<01:18, 12.77it/s]

Simulating lambdas and thetas:   0%|          | 4/1000 [00:00<01:16, 13.07it/s]

Simulating lambdas and thetas:   1%|          | 6/1000 [00:00<01:29, 11.09it/s]

Simulating lambdas and thetas:   1%|          | 8/1000 [00:00<01:24, 11.72it/s]

...

Simulating lambdas and thetas: 100%|█████████▉| 996/1000 [02:19<00:00,  6.41it/s]

Simulating lambdas and thetas: 100%|█████████▉| 998/1000 [02:19<00:00,  7.48it/s]

Simulating lambdas and thetas: 100%|█████████▉| 999/1000 [02:20<00:00,  5.87it/s]

Simulating lambdas and thetas: 100%|██████████| 1000/1000 [02:20<00:00,  6.01it/s]

Simulating lambdas and thetas: 100%|██████████| 1000/1000 [02:20<00:00,  7.13it/s]

Fitting all other methods:

In [6]:
# locart quantiles
start_time = time.time()
locart_object = LocartSplit(LambdaScore, None, alpha = alpha, is_fitted = True, split_calib = False)
locart_quantiles = locart_object.calib(model_thetas, model_lambdas, min_samples_leaf = 50)
end_time = time.time()
print(f"LOCART running time: {end_time - start_time} seconds.")

# loforest quantiles
start_time = time.time()
loforest_object = ConformalLoforest(LambdaScore, None, alpha = alpha, is_fitted = True, split_calib = False)
loforest_object.calibrate(model_thetas, model_lambdas, min_samples_leaf = 100)
end_time = time.time()
print(f"LOFOREST running time: {end_time - start_time} seconds.")

# boosting quantiles
start_time = time.time()
model = HistGradientBoostingRegressor(loss="quantile", quantile = 1 - alpha, random_state = 105)
model.fit(model_thetas, model_lambdas)
end_time = time.time()
print(f"Boosting running time: {end_time - start_time} seconds.")

LOCART running time: 0.008133888244628906 seconds.
LOFOREST running time: 0.13367795944213867 seconds.


Boosting running time: 0.35764503479003906 seconds.


Predicting for $theta$ in a relatively thin grid:


In [7]:
n_out = 45
thetas = np.linspace(0.001, 4.999, n_out)

# naive quantiles
naive_list = predict_naive_quantile(thetas, naive_quantiles)

# locart quantiles
idxs = locart_object.cart.apply(thetas.reshape(-1, 1))
list_locart_quantiles = [locart_quantiles[idx] for idx in idxs]

# loforest
loforest_cutoffs = loforest_object.compute_cutoffs(thetas.reshape(-1, 1))

# boosting
boosting_quantiles = model.predict(thetas.reshape(-1, 1))

# dictionary of quantiles
quantile_dict = {
    "naive": naive_list,
    "locart": list_locart_quantiles,
    "loforest": loforest_cutoffs,
    "boosting": boosting_quantiles,
}

Obtaining coverage data for $N = 100$:

In [8]:
cover_data = evaluate_coverage(quantile_dict, thetas)

Computing coverage for each method:   0%|          | 0/45 [00:00<?, ?it/s]

Computing coverage for each method:   2%|▏         | 1/45 [01:55<1:24:59, 115.90s/it]

Computing coverage for each method:   4%|▍         | 2/45 [03:54<1:24:09, 117.44s/it]

Computing coverage for each method:   7%|▋         | 3/45 [05:44<1:19:58, 114.26s/it]

Computing coverage for each method:   9%|▉         | 4/45 [07:30<1:15:42, 110.78s/it]

...

Computing coverage for each method:  93%|█████████▎| 42/45 [48:53<03:33, 71.24s/it]

Computing coverage for each method:  96%|█████████▌| 43/45 [50:16<02:29, 74.70s/it]

Computing coverage for each method:  98%|█████████▊| 44/45 [51:44<01:18, 78.69s/it]

Computing coverage for each method: 100%|██████████| 45/45 [53:20<00:00, 83.86s/it]

Computing coverage for each method: 100%|██████████| 45/45 [53:20<00:00, 71.12s/it]

Analising MAE and std

In [9]:
cover_data[0]

,methods,MAE,str
0,LOCART,0.017467,0.001531
1,LOFOREST,0.016933,0.001679
2,boosting,0.057200,0.007783
3,naive,0.009956,0.001218


Visualizing coverage:

In [10]:
# Melt the coverage data
coverage_data_melted = pd.melt(
    cover_data[1], id_vars=["thetas"], var_name="method", value_name="coverage"
)

# Plot the coverage as function of the thetas generated
plt.figure(figsize=(10, 6))
sns.lineplot(data=coverage_data_melted, x="thetas", y="coverage", hue="method")
plt.title("Coverage as function of the thetas generated")
plt.xlabel("Thetas")
plt.ylabel("Coverage")
plt.show()

<Figure size 3000x1800 with 1 Axes>